In [1]:
import asyncio
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
import os
from anthropic import AsyncAnthropic
from xml.sax.saxutils import escape
import re
import json

In [2]:

prompt = """\n\n
The above is a transcript of a lecture video with timestamps for each sentence in seconds.

Your task is to create a 30 second engaging and educational tiktok script for one topic in the video. 
Choose one of the more obscure and interesting topics from the transcript that most people dont know about.
The tiktok should incorporate an engaging story or example.
Do not have any emojis or hashtags in the script.
The script should be in ssml format. But do not change voices, only add pauses and emphasis.
The script should sound passionate, excited, and happy.
"""

music_prompt = """
The above is a script for a tiktok video.
Please generate a short one sentence description of the music that should be playing in the background of the video.
Include genre and mood
Example:
"A short upbeat EDM tune with a catchy melody"
"""

In [3]:
# Load transcript from data/transcript.txt
with open("data/transcript.txt", "r") as f:
    transcript = f.read()

In [4]:
async def generate_text(text, script):
    client = AsyncAnthropic(
        # This is the default and can be omitted
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    message = await client.messages.create(
        temperature=0,
        max_tokens=512,
        messages=[
            {
                "role": "user",
                "content": text + script,
            },
            {"role": "assistant", "content": "<"},
        ],
        model="claude-3-opus-20240229",
    )

    return message.content

response = asyncio.run(generate_text(transcript, prompt))

In [5]:
script = f"<{response[0].text}"

In [6]:
print(script)

<speak>
Hey everyone! Today we're diving into the fascinating world of <emphasis level="strong">tries</emphasis>! 
<break time="1s"/>
Tries may sound like something you do on a soccer field, but they're actually an incredibly powerful data structure in computer science.
<break time="1s"/>
Imagine you're organizing a massive library, but instead of books, you're dealing with words or names. That's where tries come in!
<break time="1s"/>
<emphasis level="moderate">A trie is like a tree of arrays</emphasis>, where each array represents a letter. 
To store a name like "Toad", you'd start at the root, follow the path for 'T', then 'O', 'A', and finally 'D'. 
<break time="1s"/>
The magic of tries is that they allow for <emphasis level="strong">lightning-fast searches</emphasis>. No matter how many names you have, finding "Toad" will always take the same number of steps. It's like having a super-powered librarian who can find any book instantly!
<break time="1s"/>
So the next time you're sear

In [7]:
async def generate_text(text, script):
    client = AsyncAnthropic(
        # This is the default and can be omitted
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    message = await client.messages.create(
        temperature=0,
        max_tokens=64,
        messages=[
            {
                "role": "user",
                "content": text + script,
            },
        ],
        model="claude-3-opus-20240229",
    )

    return message.content


response = asyncio.run(generate_text(script, music_prompt))
music = response[0].text

In [8]:
music

'An energetic and playful electronic track with a quirky, curious vibe.'

In [9]:
with open("data/data.json", "w") as f:
    json.dump({"script": script, "music_description": music}, f, indent=4)